In [35]:
# %pip install pandas
# %pip install utils
# %pip install pyyaml

In [36]:
import utils
from copy import deepcopy
import os
import check_constraints
from random import choice
from math import sqrt, log
from traitlets import Integer
import time

In [37]:
yaml_path = '/teamspace/studios/this_studio/Tema_IA/Tema_1/inputs/dummy.yaml'
yaml_data = utils.read_yaml_file(yaml_path)


In [38]:
zile = yaml_data['Zile']
intervale = yaml_data['Intervale']
sali = yaml_data['Sali']
profesori = yaml_data['Profesori']
studenti_materii = yaml_data['Materii']
prof_cons = {}
prof_materii = {}
total_unassigned_students = sum(studenti_materii.values())


for prof, prof_data in profesori.items():
    prof_cons[prof] = prof_data['Constrangeri']
    prof_materii[prof] = prof_data['Materii']


# sorteaza profesori dupa numarul de materii
profesori = sorted(profesori, key=lambda x: len(profesori[x]['Materii']))

sali_materii_order = {}

subject_frequency = {}
for sala, sala_data in yaml_data['Sali'].items():
    materii = sala_data.get('Materii', [])
    for materie in materii:
        subject_frequency[materie] = subject_frequency.get(materie, 0) + 1

for sala, sala_data in yaml_data['Sali'].items():
    materii = sala_data.get('Materii', [])
    sorted_materii = sorted(materii, key=lambda x: subject_frequency[x])
    sali_materii_order[sala] = sorted_materii

def generate_first_state():
    state = {zi: {eval(interval): {sala: None for sala in sali} for interval in intervale} for zi in zile}
    return state

def parse_interval(interval : str):

        intervals = interval.split('-')
        return int(intervals[0].strip()), int(intervals[1].strip())

In [39]:
class State:
    def __init__(
        self,
        timetable : dict[str, dict[tuple[int, int], dict[str, tuple[str, str]]]] | None = None,
        conflicts: int | None = None,
        ore_profesori: dict[str, int] | None = None,
        studenti_materie: dict[str, int] | None = None,
        profesor_interval: dict[str, dict[str, list[str]]] | None = None,
    ) -> None:

        self.timetable = timetable if timetable is not None \
            else generate_first_state()
        self.nconflicts = conflicts if conflicts is not None \
            else (100 * total_unassigned_students)
        self.ore_profesori = ore_profesori if ore_profesori is not None \
            else {profesor : 0 for profesor in profesori}
        self.studenti_materie = studenti_materie if studenti_materie is not None \
            else {subject : 0 for subject in studenti_materii}
        self.profesor_interval = profesor_interval if profesor_interval is not None \
            else {zi : {interval : [] for interval in intervale} for zi in zile}


    def apply_move(self, move) -> 'State':
        new_state = self.clone()
        zi, interval, sala, profesor, materie = move
        soft_conflicts = 0
        added_students = 0
        # add the move in timetable
        new_state.timetable[zi][interval][sala] = (profesor, materie)

        # profesorul nu poate preda 2 materii in acelasi interval
        interval_str = str(interval)
        if profesor not in new_state.profesor_interval[zi][interval_str]:
            new_state.profesor_interval[zi][interval_str].append(profesor)

        if new_state.ore_profesori[profesor] <= 6:
            new_state.ore_profesori[profesor] += 1

        # daca acoperire real < acoperier target
        if new_state.studenti_materie[materie] < studenti_materii[materie]:
            added_students = min(studenti_materii[materie] - new_state.studenti_materie[materie], yaml_data['Sali'][sala]['Capacitate'])
            new_state.studenti_materie[materie] += added_students

        #### SOFT CONSTRAINTS ####

        for const in prof_cons[profesor]:
            if const[0] != '!':
                continue
            else:
                const = const[1:]
                if zi in const:
                    soft_conflicts += 1
                elif '-' in const:
                    start, end = parse_interval(const)
                    if start != end - 2:
                        intervals = [(i, i + 2) for i in range(start, end, 2)]
                    else:
                        intervals = [(start, end)]
                    if interval in intervals:
                        soft_conflicts += 1

        # calculate the conflicts
        new_state.nconflicts = new_state.nconflicts + soft_conflicts - 100 * added_students

        return new_state

    def conflicts(self) -> int:
        '''
        Întoarce numărul de conflicte din această stare.
        '''
        return self.nconflicts

    def is_final(self) -> bool:
        '''
        Întoarce True dacă este stare finală.
        '''
        added_students = sum(self.studenti_materie.values())
        if added_students == total_unassigned_students:
            return True
        
        if self.get_next_states() == []:
            return True
        
        return False

    def get_next_states(self):

        available_moves = []
        for zi in self.timetable:
            for interval in self.timetable[zi]:
                for sala in self.timetable[zi][interval]:
                    if self.timetable[zi][interval][sala] is None:
                        for materie in sali_materii_order[sala]:
                            if self.studenti_materie[materie] < studenti_materii[materie]:
                                for profesor in profesori:
                                    if materie in prof_materii[profesor] and profesor not in self.profesor_interval[zi][str(interval)] and self.ore_profesori[profesor] <= 6:
                                        available_moves.append(self.apply_move((zi, interval, sala, profesor, materie)))

        return available_moves

    def display(self):
        return(utils.pretty_print_timetable_aux_zile(self.timetable, yaml_path))

    def clone(self):
        return State(deepcopy(self.timetable), deepcopy(self.nconflicts), deepcopy(self.ore_profesori), deepcopy(self.studenti_materie), deepcopy(self.profesor_interval))


In [40]:
Result = tuple[bool, int, int, State]
def hill_climbing(initial: State, max_iters: int = 1000) -> Result:
    iters, states = 0, 0
    state = initial.clone()

    while iters < max_iters:
        iters += 1

        next_states = state.get_next_states()
        states += len(next_states)

        best_neighbor = None
        min_conflicts = state.conflicts()

        for neighbor in next_states:
            conflicts = neighbor.conflicts()
            if conflicts < min_conflicts:
                best_neighbor = neighbor
                min_conflicts = conflicts

        if best_neighbor is None:
            break

        state = best_neighbor
    return state.is_final(), iters, states, state

In [41]:
# result = hill_climbing(State())
# final_state = result[-1]
# print(final_state.display())
# print(check_constraints.check_mandatory_constraints(final_state.timetable, yaml_data))
# print(check_constraints.check_optional_constraints(final_state.timetable, yaml_data))
# print(str(result[1]))
# print(str(result[2]))
start_time = time.time()
result = hill_climbing(State())
end_time = time.time()

# Extract the final state from the result
final_state = result[-1]

# Display the final timetable
final_state.display()

# Check mandatory and optional constraints on the final timetable
mandatory_constraints_passed = check_constraints.check_mandatory_constraints(final_state.timetable, yaml_data)
optional_constraints_passed = check_constraints.check_optional_constraints(final_state.timetable, yaml_data)

# Print the results
print("Mandatory Constraints Passed:", mandatory_constraints_passed)
print("Optional Constraints Passed:", optional_constraints_passed)

# Print the time taken for execution
execution_time = end_time - start_time
print("Execution Time:", execution_time)

# Write the results to a file
with open("orar_mic_exact.txt", "w") as file:
    file.write("Hill Climbing Algorithm\n")
    file.write("Test File: " + yaml_path + "\n\n")
    file.write("Number of iterations: " + str(result[1]) + "\n")
    file.write("Number of states explored: " + str(result[2]) + "\n")
    file.write("Mandatory Constraints Passed: " + str(mandatory_constraints_passed) + "\n")
    file.write("Optional Constraints Passed: " + str(optional_constraints_passed) + "\n")
    file.write("Execution Time: " + str(execution_time) + " seconds\n")
    file.write(final_state.display())

Mandatory Constraints Passed: 0
Optional Constraints Passed: 0
Execution Time: 0.05076098442077637


In [42]:
N = 'N'
Q = 'Q'
STATE = 'state'
PARENT = 'parent'
ACTIONS = 'actions'


def init_node(state, parent = None):
    return {N: 0, Q: 0, STATE: state, PARENT: parent, ACTIONS: {}}

CP = 1.0 / sqrt(2.0)

def select_action(node, c = CP):
    N_node = node[N]
    maxNode = None
    maxValue = float('-inf')
    for key, value in node[ACTIONS].items():
        if maxValue < value[Q] / value[N] + c * sqrt(2 * log(N_node) / value[N]):
          maxValue = value[Q] / value[N] + c * sqrt(2 * log(N_node) / value[N])
          maxNode = key

    return maxNode

In [43]:
def mcts(state0: State, budget, tree, last_action=None, visited_nodes=0):
    if tree and last_action in tree[ACTIONS]:
        tree = tree[ACTIONS][last_action]
    else:
        tree = init_node(state0)

    # Simulare
    for x in range(budget):
        node = tree
        # parcurg toti vecinii si verific daca sunt vizitati 
        while all(action in node[ACTIONS] for action in state0.get_next_states()) and (not state0.is_final()):
            # si daca sunt vizitati, aleg actiunea cu cel mai mare Q/N
            action = select_action(node)
            state0 = action
            node = node[ACTIONS][action]
        
        # Expansiune
        #Dacă nodul curent nu este final și mai sunt acțiuni posibile care nu au fost încă explorate,
        # se alege una dintre aceste acțiuni și se inițializează un nod nou pentru starea rezultată.
        if node and (not state0.is_final()):
            action = choice([a for a in state0.get_next_states() if a not in node[ACTIONS]])
            new_state = action
            node = init_node(new_state, node)
            node[PARENT][ACTIONS][action] = node

        state = node[STATE]
        # Se simulează jocul începând de la nodul curent până se ajunge la un nod final.
        # Se alege aleatoriu una dintre acțiunile disponibile și se actualizează starea.
        while not state.is_final():
            visited_nodes += 1
            actions = state.get_next_states()
            action = choice(actions)
            state = action

        reward = -state.conflicts()
        # Odată ce s-a ajuns la o stare finală, se calculează recompensa
        # Recompensa este propagată înapoi prin arbore, actualizând numărul de vizite și
        # valoarea totală a recompenselor pentru fiecare nod de pe drumul parcurs.
        while node:
            node[N] += 1
            node[Q] += reward
            node = node[PARENT]
    # După ce toate simulările s-au încheiat, se alege acțiunea finală folosind funcția select_action
    # cu un parametru care indică explorarea zero (pentru a alege cea mai bună acțiune cunoscută).
    # Dacă nu există acțiuni posibile, returnează prima acțiune disponibilă sau un nod inițializat cu starea inițială.
    if tree:
        final_action = select_action(tree, 0.0)
        return (final_action, tree[ACTIONS][final_action], visited_nodes)
    if state0.get_next_states(state0):
        return (state0.get_next_states(state0)[0], init_node(), visited_nodes)
    return (0, init_node(state0), visited_nodes)


In [44]:
memory = None

# Se desfășoară jocul
state = State()
last_action = None
iteration = 0
totatal_visited_nodes = 0

start_time = time.time()
while state and not state.is_final():
    visited_nodes = 0
    (action, memory, visited_nodes) = mcts(state, 50, memory, last_action)
    iteration += 1
    state = action
    last_action = action
    totatal_visited_nodes += visited_nodes
end_time = time.time()
print(state.display())
print(check_constraints.check_mandatory_constraints(state.timetable, yaml_data))
print(check_constraints.check_optional_constraints(state.timetable, yaml_data))


with open("results.txt", "w") as file:
    file.write("Test File: " + yaml_path + "\n\n")
    file.write("Number of iterations: " + str(iteration) + "\n")
    file.write("Number of states explored: " + str(totatal_visited_nodes) + "\n")
    file.write("Execution Time: " + str(end_time - start_time) + " seconds\n")
    file.write("Mandatory Constraints Passed: " + str(check_constraints.check_mandatory_constraints(state.timetable, yaml_data)) + "\n")
    file.write("Optional Constraints Passed: " + str(check_constraints.check_optional_constraints(state.timetable, yaml_data)) + "\n")
    file.write(state.display())

|           Interval           |             Luni             |             Marti            |           Miercuri           |              Joi             |            Vineri            |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|            8 - 10            |      MS : (EG324 - RG)       |      MS : (EG324 - CD)       |      IA : (EG324 - PF)       |
|                              |      DS : (EG390 - CD)       |      EG390 - goala           |      EG390 - goala           |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|            10 - 12           |      EG324 - goala           |      EG324 - goala           |      IA : (EG324 - PF)       |
|                              |      EG390 - goala       

In [45]:
# import os
# import matplotlib.pyplot as plt

# def extract_data(file_path):
#     with open(file_path, 'r') as file:
#         lines = file.readlines()
        
#         # Initialize variables
#         number_of_iterations = 0
#         number_of_states_explored = 0
#         mandatory_constraints_passed = 0
#         optional_constraints_passed = 0
#         execution_time = 0.0
        
#         # Parse data from lines
#         for line in lines:
#             if line.startswith("Number of iterations:"):
#                 number_of_iterations = int(line.split(':')[-1].strip())
#             elif line.startswith("Number of states explored:"):
#                 number_of_states_explored = int(line.split(':')[-1].strip())
#             elif line.startswith("Mandatory Constraints Passed:"):
#                 mandatory_constraints_passed = int(line.split(':')[-1].strip())
#             elif line.startswith("Optional Constraints Passed:"):
#                 optional_constraints_passed = int(line.split(':')[-1].strip())
#             elif line.startswith("Execution Time:"):
#                 execution_time = float(line.split(':')[-1].strip().split()[0])  # Extracting only the time value
        
#         data = {
#             'Number of iterations': number_of_iterations,
#             'Number of states explored': number_of_states_explored,
#             'Mandatory Constraints Passed': mandatory_constraints_passed,
#             'Optional Constraints Passed': optional_constraints_passed,
#             'Execution Time': execution_time
#         }
        
#     return data

# def plot_comparison(data_dict):
#     fig, axs = plt.subplots(3, 2, figsize=(12, 12))
#     metrics = ['Number of iterations', 'Number of states explored', 
#                'Mandatory Constraints Passed', 'Optional Constraints Passed', 'Execution Time']
#     for i, metric in enumerate(metrics):
#         if metric == 'Execution Time':
#             axs[i//2, i%2].bar(data_dict.keys(), [data[metric] for data in data_dict.values()], color='orange')
#         else:
#             axs[i//2, i%2].bar(data_dict.keys(), [data[metric] for data in data_dict.values()])
#         axs[i//2, i%2].set_title(metric)
#         axs[i//2, i%2].set_ylabel(metric)
#         axs[i//2, i%2].set_xlabel('File')
#         axs[i//2, i%2].tick_params(axis='x', rotation=45)
#     plt.tight_layout()
#     plt.show()

# folder_path = './HillClimbing'
# file_names = [file for file in os.listdir(folder_path) if file.endswith('.txt')]

# data_dict = {}
# for file_name in file_names:
#     file_path = os.path.join(folder_path, file_name)
#     data = extract_data(file_path)
#     data_dict[file_name] = data

# plot_comparison(data_dict)


In [46]:
# import matplotlib.pyplot as plt

# # Funcție pentru extragerea datelor din fișier
# def extract_data(file_path):
#     with open(file_path, 'r') as file:
#         lines = file.readlines()
#         data = {}
#         for line in lines:
#             if line.startswith("Number of iterations:"):
#                 data['Number of iterations'] = int(line.split(':')[-1].strip())
#             elif line.startswith("Number of states explored:"):
#                 data['Number of states explored'] = int(line.split(':')[-1].strip())
#             elif line.startswith("Mandatory Constraints Passed:"):
#                 data['Mandatory Constraints Passed'] = int(line.split(':')[-1].strip())
#             elif line.startswith("Optional Constraints Passed:"):
#                 data['Optional Constraints Passed'] = int(line.split(':')[-1].strip())
#             elif line.startswith("Execution Time:"):
#                 data['Execution Time'] = float(line.split(':')[-1].strip().split()[0])  # Extracting only the time value
#     return data

# # Extragem datele pentru fiecare algoritm
# hill_climbing_data = extract_data('./HillClimbing/dummy.txt')  # Datele pentru Hill Climbing
# mcts_data = extract_data('./MCTS/dummy.txt')                    # Datele pentru MCTS

# # Definim lista de metrici
# metrics = ['Number of iterations', 'Number of states explored', 
#            'Mandatory Constraints Passed', 'Optional Constraints Passed', 'Execution Time']

# # Generăm grafice pentru fiecare metrică
# for metric in metrics:
#     plt.figure(figsize=(8, 6))
#     plt.bar(['Hill Climbing', 'MCTS'], [hill_climbing_data[metric], mcts_data[metric]], color=['blue', 'green'])
#     plt.title(metric)
#     plt.ylabel(metric)
#     plt.show()
